### Jira Implementation
Jira can be used to track and measure these metrics in the following ways:

- **Defects:** Use Jira issues to track defects. The status of the issue can indicate whether the defect was found pre-release (status: closed) or post-release (status: open).
- **Time Tracking:** Use Jira's built-in time tracking features to measure MTTA and MTTR. The "Created" timestamp can indicate when a defect was reported, and the "Resolved" timestamp can indicate when it was resolved.
- **System Failures:** Use Jira issues to track system failures. The "Created" timestamp can indicate the time of each failure, and the time between failures can be calculated from these timestamps.
- **Reporting:** Use Jira's built-in reporting features to generate reports on these metrics. These reports can be used to monitor performance, identify trends, and make data-driven decisions.

### Relationship with CSAT and SLA
These metrics can have a significant impact on Customer Satisfaction (CSAT) and Service Level Agreements (SLAs):

- **CSAT:** High DER can lead to lower CSAT as more defects are found by customers post-release. Faster MTTA and MTTR can lead to higher CSAT as customers see that their reports are being addressed quickly. Higher MTBF (i.e., fewer failures) can also lead to higher CSAT.
- **SLA:** High DER can lead to SLA breaches if the time to fix post-release defects exceeds the agreed SLA. Faster MTTA and MTTR can help meet SLAs for acknowledging and resolving reported defects. Higher MTBF can help meet SLAs for system uptime.

In [ ]:
!pip install jira

In [ ]:
from jira import JIRA
import pandas as pd

In [ ]:
options = {
    'server': 'https://your-jira-server.com',
}

oauth_creds = {
    'access_token': 'YOUR_ACCESS_TOKEN',
    'access_token_secret': 'YOUR_ACCESS_TOKEN_SECRET',
    'consumer_key': 'YOUR_CONSUMER_KEY',
    'key_cert': 'YOUR_RSA_PRIVATE_KEY',
}

jira = JIRA(options=options, oauth=oauth_creds)

In [ ]:
# Query Jira for defects (e.g., issues with issue type 'Bug')
defects_query = 'issuetype = Bug AND project = YOUR_PROJECT'
defects = jira.search_issues(defects_query)

# Extract relevant information from defects
defects_data = []
for defect in defects:
    issue_key = defect.key
    created_time = defect.fields.created
    resolved_time = defect.fields.resolutiondate
    # Assuming 'GitHub Commits' is a custom field storing commit messages from Smart Commits
    commits = defect.fields.customfield_GitHubCommits
    defects_data.append((issue_key, created_time, resolved_time, commits))

# Create a Pandas dataframe
defects_df = pd.DataFrame(defects_data, columns=['Issue Key', 'Created Time', 'Resolved Time', 'Commits'])
defects_df.head()

**Please note that this code is intended to be run in your local environment where you have access to both Jira and GitHub. You'll need to provide the GitHub token and repository name as inputs, and you may need to make adjustments based on your specific use case and data structure.**



In [ ]:
# Install PyGithub library (Run this in your local environment)
!pip -q install PyGithub

from github import Github
import re
import pandas as pd
import matplotlib.pyplot as plt

# Function to extract Jira issue keys from commit messages
def extract_issue_keys(message):
    return re.findall(r'[A-Z]+-\d+', message)

# Inputs for GitHub token and repository name
github_token = input("Enter your GitHub token: ")
repo_name = input("Enter the repository name (e.g., 'user/repo'): ")

# Authenticate with GitHub
g = Github(github_token)

# Get the repository
repo = g.get_repo(repo_name)

# Get commits and extract relevant information
commits_data = []
for commit in repo.get_commits():
    commit_message = commit.commit.message
    commit_time = commit.commit.author.date
    issue_keys = extract_issue_keys(commit_message)
    commits_data.append((commit_message, commit_time, issue_keys))

# Create a Pandas dataframe for commits
commits_df = pd.DataFrame(commits_data, columns=['Commit Message', 'Commit Time', 'Issue Keys'])

# Function to flatten issue keys and merge with defects
def map_commits_to_defects(defects_df, commits_df):
    mapping_data = []
    for index, row in commits_df.iterrows():
        for issue_key in row['Issue Keys']:
            mapping_data.append((row['Commit Message'], row['Commit Time'], issue_key))
    mapping_df = pd.DataFrame(mapping_data, columns=['Commit Message', 'Commit Time', 'Issue Key'])
    merged_df = pd.merge(defects_df, mapping_df, on='Issue Key', how='left')
    return merged_df

# Map commits to defects
mapped_data = map_commits_to_defects(defects_df, commits_df)

# Function to plot the timeline
def plot_timeline(mapped_data):
    plt.figure(figsize=(12, 6))
    for index, row in mapped_data.iterrows():
        plt.plot([row['Created Time'], row['Commit Time'], row['Resolved Time']], [index, index, index], marker='o')
        plt.text(row['Commit Time'], index, row['Commit Message'], fontsize=8)
    plt.xlabel('Time')
    plt.ylabel('Defects and Commits')
    plt.title('Timeline of Jira Defects and GitHub Commits')
    plt.show()

# Plot the timeline
plot_timeline(mapped_data)